# Optuna Study – Bi‑LSTM Hyper‑Parameters

In [1]:
import optuna, os, tensorflow as tf, numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-16 00:31:16.251022: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 00:31:16.255836: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 00:31:16.270710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750033876.293132   40525 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750033876.301423   40525 cuda_blas.cc:14

In [2]:
def get_data(MAX_VOCAB=25_000, MAX_LEN=120):
    df = load_dataset("liar", split="train").to_pandas()[["statement","label"]]
    df.columns = ["text","target"]
    df.target = df.target.apply(lambda x: 0 if x in [0,1,2] else 1)
    tok = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
    tok.fit_on_texts(df.text)
    X = pad_sequences(tok.texts_to_sequences(df.text), maxlen=MAX_LEN, padding="post")
    y = df.target.values
    return tok, *train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [3]:
tok, X_tr, X_te, y_tr, y_te = get_data()

def objective(trial):
    emb_dim    = trial.suggest_categorical("emb_dim",[64,128,256])
    lstm_units = trial.suggest_int("lstm_units",32,128,step=32)
    dropout    = trial.suggest_float("dropout",0.1,0.5,step=0.1)
    lr         = trial.suggest_float("lr",1e-4,5e-3,log=True)

    model = Sequential([
        Embedding(len(tok.word_index)+1, emb_dim, mask_zero=True),
        Bidirectional(LSTM(lstm_units)),
        Dropout(dropout),
        Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), metrics=["accuracy"])
    hist = model.fit(X_tr, y_tr, epochs=3, batch_size=128, validation_split=0.2, verbose=0)
    val_acc = max(hist.history["val_accuracy"])
    tf.keras.backend.clear_session()
    return val_acc

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)
study.best_params, study.best_value

[I 2025-06-16 01:07:34,197] A new study created in memory with name: no-name-1aad02be-8a3b-4c55-96a4-be47d3a5a478
Best trial: 0. Best value: 0.592818:   5%|▌         | 1/20 [01:05<20:46, 65.63s/it]

[I 2025-06-16 01:08:39,828] Trial 0 finished with value: 0.592818021774292 and parameters: {'emb_dim': 128, 'lstm_units': 96, 'dropout': 0.5, 'lr': 0.0006587530964898243}. Best is trial 0 with value: 0.592818021774292.


Best trial: 0. Best value: 0.592818:  10%|█         | 2/20 [02:22<21:36, 72.04s/it]

[I 2025-06-16 01:09:56,348] Trial 1 finished with value: 0.592818021774292 and parameters: {'emb_dim': 64, 'lstm_units': 128, 'dropout': 0.5, 'lr': 0.0004301893639666093}. Best is trial 0 with value: 0.592818021774292.


Best trial: 2. Best value: 0.600122:  15%|█▌        | 3/20 [03:12<17:32, 61.92s/it]

...

Best trial: 10. Best value: 0.601339:  95%|█████████▌| 19/20 [19:34<00:45, 45.48s/it]

[I 2025-06-16 01:27:08,925] Trial 18 finished with value: 0.5952525734901428 and parameters: {'emb_dim': 256, 'lstm_units': 32, 'dropout': 0.4, 'lr': 0.0014181969098628674}. Best is trial 10 with value: 0.6013390421867371.


Best trial: 10. Best value: 0.601339: 100%|██████████| 20/20 [20:29<00:00, 61.45s/it]

[I 2025-06-16 01:28:03,235] Trial 19 finished with value: 0.5952525734901428 and parameters: {'emb_dim': 128, 'lstm_units': 64, 'dropout': 0.2, 'lr': 0.002748261086611754}. Best is trial 10 with value: 0.6013390421867371.


({'emb_dim': 64, 'lstm_units': 64, 'dropout': 0.1, 'lr': 0.00490707802464229},
 0.6013390421867371)

In [9]:
best = study.best_params
emb_dim, lstm_units, dropout, lr = best["emb_dim"],best["lstm_units"],best["dropout"],best["lr"]
tok_full, X_tr, X_te, y_tr, y_te = get_data()
model = Sequential([
    Embedding(len(tok_full.word_index)+1, emb_dim, mask_zero=True),
    Bidirectional(LSTM(lstm_units)),
    Dropout(dropout),
    Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), metrics=["accuracy"])
model.fit(X_tr, y_tr, epochs=5, batch_size=128, validation_split=0.1, verbose=0)
os.makedirs("models", exist_ok=True)
model.save("models/bi_lstm_fake_news.h5")

In [10]:
import pickle
with open("models/tokenizer.pkl", "wb") as f:
    pickle.dump(tok_full, f)
print("🚀 Saved tuned model to models/bi_lstm_fake_news.h5")

🚀 Saved tuned model to models/bi_lstm_fake_news.h5
